# Minimize Malware Spread

You are given a network of n nodes represented as an n x n adjacency matrix graph, where the ith node is directly connected to the jth node if graph[i][j] == 1.

Some nodes initial are initially infected by malware. Whenever two nodes are directly connected, and at least one of those two nodes is infected by malware, both nodes will be infected by malware. This spread of malware will continue until no more nodes can be infected in this manner.

Suppose M(initial) is the final number of nodes infected with malware in the entire network after the spread of malware stops. We will remove exactly one node from initial.

Return the node that, if removed, would minimize M(initial). If multiple nodes could be removed to minimize M(initial), return such a node with the smallest index.

Note that if a node was removed from the initial list of infected nodes, it might still be infected later due to the malware spread.

**Example 1:**

Input: graph = [[1,1,0],[1,1,0],[0,0,1]], initial = [0,1]
Output: 0

**Example 2:**

Input: graph = [[1,0,0],[0,1,0],[0,0,1]], initial = [0,2]
Output: 0

**Example 3:**

Input: graph = [[1,1,1],[1,1,1],[1,1,1]], initial = [1,2]
Output: 1

**Constraints:**

- n == graph.length
- n == graph[i].length
- 2 <= n <= 300
- graph[i][j] is 0 or 1.
- graph[i][j] == graph[j][i]
- graph[i][i] == 1
- 1 <= initial.length <= n
- 0 <= initial[i] <= n - 1
- All the integers in initial are unique.

In [1]:
class UnionFind:
    def __init__(self, size):
        self.parent = list(range(size))
        self.rank = [1] * size
    
    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]
    
    def union(self, x, y):
        rootX = self.find(x)
        rootY = self.find(y)
        if rootX != rootY:
            if self.rank[rootX] > self.rank[rootY]:
                self.parent[rootY] = rootX
            elif self.rank[rootX] < self.rank[rootY]:
                self.parent[rootX] = rootY
            else:
                self.parent[rootY] = rootX
                self.rank[rootX] += 1

def minMalwareSpread(graph, initial):
    n = len(graph)
    
    # Step 1: Use Union-Find to group nodes into components
    uf = UnionFind(n)
    
    for i in range(n):
        for j in range(n):
            if graph[i][j] == 1:
                uf.union(i, j)
    
    # Step 2: Count the size of each component
    component_size = [0] * n
    for i in range(n):
        root = uf.find(i)
        component_size[root] += 1
    
    # Step 3: Count how many initial infected nodes are in each component
    infected_in_component = [0] * n
    for node in initial:
        root = uf.find(node)
        infected_in_component[root] += 1
    
    # Step 4: Try removing each node in initial
    best_node = -1
    max_saved = -1
    
    for node in initial:
        root = uf.find(node)
        
        # If this component has exactly one infected node, we can save the component
        if infected_in_component[root] == 1:
            saved = component_size[root]
        else:
            saved = 0